In [1]:
# Scientific libraries		              
import numpy as np		              
import scipy as sp		              
import pandas as pd		              
import json		              
import missingno as msno		              

# Loading Plotting Modules		              
import matplotlib		              
import matplotlib.pyplot as plt		              
import seaborn as sns		              
%matplotlib inline		              
import chart_studio.plotly as py		              
import plotly.figure_factory as ff		              
import plotly.graph_objects as go		              
import plotly.express as px		              
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot		              
init_notebook_mode(connected=True)		              
		              
import os		              
import gc

In [2]:
# Setting Data Frame Options                      
pd.set_option('display.max_rows', 40)                      
pd.set_option('display.max_columns', 50)                      
pd.set_option('display.width', 100)                      
pd.set_option('display.max_colwidth', 100)                      

# Setting Plot Configuration                      
sns.set(rc={'figure.figsize':(19,11)}, style = 'white')

In [3]:
# test = pd.read_csv('../../../data/test.csv')          
test = pd.read_csv('../../eda/outputs/imputed_test_mfc2.csv')         
m, n = test.shape
print('Data Frame: {} x {}'.format(m,n))                
test.head()

Data Frame: 890 x 79


,existence expectancy index,existence expectancy at birth,Gross income per capita,Income Index,Expected years of education (galactic years),Mean years of education (galactic years),Intergalactic Development Index (IDI),Education Index,"Intergalactic Development Index (IDI), Rank",Population using at least basic drinking-water services (%),Population using at least basic sanitation services (%),Gross capital formation (% of GGP),"Population, total (millions)","Population, urban (%)","Mortality rate, under-five (per 1,000 live births)","Mortality rate, infant (per 1,000 live births)",Old age dependency ratio (old age (65 and older) per 100 creatures (ages 15-64)),"Population, ages 15–64 (millions)","Population, ages 65 and older (millions)","Life expectancy at birth, male (galactic years)","Life expectancy at birth, female (galactic years)","Population, under age 5 (millions)",Young age (0-14) dependency ratio (per 100 creatures ages 15-64),"Adolescent birth rate (births per 1,000 female creatures ages 15-19)",Total unemployment rate (female to male ratio),...,"Estimated gross galactic income per capita, female",Rural population with access to electricity (%),Domestic credit provided by financial sector (% of GGP),"Population with at least some secondary education, female (% ages 25 and older)","Population with at least some secondary education, male (% ages 25 and older)",Gross fixed capital formation (% of GGP),"Remittances, inflows (% of GGP)",Population with at least some secondary education (% ages 25 and older),Intergalactic inbound tourists (thousands),"Gross enrolment ratio, primary (% of primary under-age population)","Respiratory disease incidence (per 100,000 people)",Interstellar phone subscriptions (per 100 people),"Interstellar Data Net users, total (% of population)",Current health expenditure (% of GGP),"Intergalactic Development Index (IDI), female","Intergalactic Development Index (IDI), male",Gender Development Index (GDI),"Intergalactic Development Index (IDI), female, Rank","Intergalactic Development Index (IDI), male, Rank",Adjusted net savings,"Creature Immunodeficiency Disease prevalence, adult (% ages 15-49), total",Private galaxy capital flows (% of GGP),Gender Inequality Index (GII),galactic year,galaxy
0,0.456086,51.562543,12236.576447,0.593325,10.414164,10.699072,0.547114,0.556267,232.621842,105.193088,64.241392,17.418350,709.974500,73.297380,146.341817,100.005259,12.562363,489.982611,53.149371,56.249570,60.594004,82.932522,106.521709,169.754218,2.509936,...,13681.345449,65.920807,96.448861,59.897899,70.066364,16.631279,11.064357,62.046027,99833.452672,104.199637,398.486742,87.291073,33.699239,7.830928,0.578841,0.648238,0.937948,187.932285,190.203266,12.229105,7.611505,24.146867,0.834866,1007012,KK98 77
1,0.529835,57.228262,3431.883825,0.675407,7.239485,5.311122,0.497688,0.409969,247.580771,55.730638,46.217440,33.027886,663.067688,52.484030,166.038771,108.221474,13.200530,514.865394,54.980664,56.843449,60.719106,73.546787,113.037027,183.668653,2.449988,...,13112.402114,40.524312,81.914200,35.913800,39.242603,29.799164,9.411752,38.035978,98273.087328,79.498161,352.781610,63.668477,31.235817,7.493285,0.465698,0.564255,0.882829,191.034292,199.069702,18.083477,6.588888,24.547531,0.880047,1007012,Reticulum III
2,0.560976,59.379539,27562.914252,0.594624,11.774890,5.937797,0.544744,0.486167,249.798771,58.755900,48.897493,31.613362,679.824897,52.027110,154.631808,100.317484,13.144451,518.007633,55.026892,60.597547,61.235156,76.754857,113.361712,178.938322,2.437677,...,13935.601262,39.724189,82.407018,37.618966,47.066875,29.734731,9.571031,43.073675,96229.282018,113.616580,338.213199,69.731913,31.865079,7.822061,0.539204,0.589167,0.939487,187.440453,193.334980,16.383094,7.047281,25.892463,0.832374,1008016,Reticulum III
3,0.565910,59.952390,20352.232905,0.837700,11.613621,10.067882,0.691641,0.523441,211.505060,70.176431,61.254104,34.399007,703.504462,66.914935,138.682272,85.920319,12.103836,466.974474,52.12288

    Potential for increase in the Index = -np.log(Index+0.01)+3

In [4]:
# RMSE Train: 0.001800391617117943
# RMSE Valid: 0.052504038023260875

# RMSE Train:  0.00438560
# RMSE Valid:  0.01734807	Ratio (val/train):  3.95569

preds = pd.read_csv('../outputs/y_pred_lgb_knn_20200621_005107.csv')
preds['opt_pred'] = 0
preds['eei'] = test['existence expectancy index']
preds['p_increase'] = -np.log(preds['y_pred'] + 0.01) + 3
preds['p_increase2'] = preds['p_increase'] ** 2
preds

,Index,y_pred,opt_pred,eei,p_increase,p_increase2
0,0,0.045021,0,0.456086,5.900041,34.810485
1,1,0.042510,0,0.529835,5.946748,35.363818
2,2,0.041625,0,0.560976,5.963750,35.566317
3,3,0.040692,0,0.565910,5.981979,35.784078
4,4,0.038159,0,0.588274,6.033244,36.400031
...,...,...,...,...,...,...
885,885,0.048458,0,0.898146,5.839438,34.099041
886,886,0.056439,0,0.898146,5.711476,32.620961
887,887,0.076542,0,0.898146,5.447120,29.671113
888,888,0.067630,0,0.898146,5.555799,30.866901


### Optimization

We will try to maximize sum of likely increase index for all galaxies, which for one galaxy is described as follows:

    Likely increase in the Index = extra energy * Potential for increase in the Index ** 2 / 1000

with respect to the following constraints.

* in total there are 50000 zillion DSML available for allocation
* no galaxy should be allocated more than 100 zillion DSML or less than 0 zillion DSML
* galaxies with low existence expectancy index below 0.7 should be allocated at least 10% of the total energy available

In [5]:
from scipy.optimize import minimize

In [6]:
preds['is_low'] = False
preds.loc[preds.eei < 0.7, 'is_low'] = True
preds.sort_values(by='is_low', ascending=False, inplace=True)
preds.reset_index(drop=True, inplace=True)
preds

,Index,y_pred,opt_pred,eei,p_increase,p_increase2,is_low
0,0,0.045021,0,0.456086,5.900041,34.810485,True
1,50,0.042187,0,0.676239,5.952931,35.437389,True
2,36,0.042009,0,0.652121,5.956329,35.477859,True
3,37,0.029994,0,0.656685,6.219018,38.676182,True
4,38,0.034667,0,0.656738,6.108531,37.314155,True
...,...,...,...,...,...,...,...
885,342,0.046492,0,0.903199,5.873656,34.499840,False
886,343,0.037117,0,0.903945,6.055131,36.664611,False
887,344,0.036500,0,0.903964,6.068313,36.824424,False
888,345,0.052883,0,0.904205,5.766486,33.252357,False


In [7]:
def objective(energy, p2):
    return -sum((energy * p2) / 1000)

In [8]:
%%time
m = test.shape[0]
bounds = [(0,100) for _ in range(m)]

# low_p2 = preds.loc[preds['eei'] < 0.7,'p_increase2'].copy().tolist()
# high_p2 = preds.loc[preds['eei'] >= 0.7,'p_increase2'].copy().tolist()
m_low = preds.loc[preds['eei'] < 0.7,'p_increase2'].shape[0]
p2 = preds['p_increase2']

cons = (
    {"type": "ineq", "fun": lambda x: 49999.999 - sum(x)},
    {"type": "ineq", "fun": lambda x: sum(x[:m_low]) - 5000.001}
)

history = minimize(objective, x0=np.zeros(m),
                   args=(p2),
                   bounds=bounds, constraints=cons)

Wall time: 2min 10s


In [9]:
preds['opt_pred'] = history.x
preds

,Index,y_pred,opt_pred,eei,p_increase,p_increase2,is_low
0,0,0.045021,53.631609,0.456086,5.900041,34.810485,True
1,50,0.042187,63.264504,0.676239,5.952931,35.437389,True
2,36,0.042009,95.931278,0.652121,5.956329,35.477859,True
3,37,0.029994,98.888982,0.656685,6.219018,38.676182,True
4,38,0.034667,100.000000,0.656738,6.108531,37.314155,True
...,...,...,...,...,...,...,...
885,342,0.046492,76.087962,0.903199,5.873656,34.499840,False
886,343,0.037117,93.715878,0.903945,6.055131,36.664611,False
887,344,0.036500,94.999436,0.903964,6.068313,36.824424,False
888,345,0.052883,65.978217,0.904205,5.766486,33.252357,False


In [10]:
print('Total maximum likely increase:', -objective(history.x, p2))
print('Energy allocated (total):', preds.opt_pred.sum())
print('Energy allocated (low):', preds[preds.is_low].opt_pred.sum())

Total maximum likely increase: 1742.445319377199
Energy allocated (total): 49999.99899667595
Energy allocated (low): 5620.091360804606


In [11]:
submission_file = preds[['Index', 'y_pred', 'opt_pred']]
submission_file.rename(columns={
    "Index": "index",
    "y_pred": "pred",
}, inplace=True)
submission_file.sort_values(by='index', ascending=True, inplace=True)
submission_file

,index,pred,opt_pred
0,0,0.045021,53.631609
31,1,0.042510,54.405068
36,2,0.041625,99.520995
38,3,0.040692,99.517378
39,4,0.038159,99.566534
...,...,...,...
343,885,0.048458,72.780279
344,886,0.056439,60.799191
345,887,0.076542,36.748716
346,888,0.067630,46.485685


In [12]:
import datetime
ts_now = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')
submission_file.to_csv(f'../../../submissions/submitted/submission_19_knn_{ts_now}.csv', index=False)